In [2]:
import pandas as pd
import requests
import spotipy.util as util
from config import *
from tqdm import tqdm
import numpy as np

In [111]:
def get_token(user: str, 
              client_id: str,
              client_secret: str,
              redirect_uri: str,
              scope: str) -> str:
  
    token = util.prompt_for_user_token(user,scope,
                                               client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri)
    return token

In [112]:
token = get_token(username, client_id, client_secret, redirect_uri, scope)

In [113]:
audi_feat_columns = set([
    "uri",
    "danceability",
    "energy", 
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
])

In [114]:
def get_song_id(track_name):
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' +token,
    }

    params = {
        'q': 'track:' + track_name,
        'type': 'track',
    }

    response = requests.get('https://api.spotify.com/v1/search', params=params, headers=headers)
    
    # Response.items has many levels:
    #      - First, it contains just a list (list(response.json().items())[0])
    #      - Then, the first item in the list is "Tracks" and the second item is our dictionary ([1])
    #      - We need to access our items which is again a list
    #      - Finally, we need to get our id
    items = list(response.json().items())[0][1]["items"][0]["id"]
    
    return items
    

In [115]:
songs_ids = {}

In [116]:
def get_all_songs_ids(my_history, track_name_column):
    failed_songs = []
    for song in tqdm(my_history[track_name_column].unique()):
        try:
            if song not in songs_ids:
                songs_ids[song] = get_song_id(song)
        except:
            failed_songs.append(song)
            continue

    my_history["uri"] = my_history[track_name_column].map(songs_ids)
    
    return my_history, failed_songs

In [117]:
def get_auido_feats_by_ids(list_of_ids):
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
     'Authorization': f'Bearer ' +token,
    }

    params = {
        'ids': list_of_ids,
    }

    response = requests.get('https://api.spotify.com/v1/audio-features', params=params, headers=headers)
    
    return response

In [118]:
def get_request_chunks(my_uris):
    max_requests, ids_count = 100, 0
    
    requests_chunks = []
    concat_uris = ""
    
    for uri in my_uris:
        concat_uris += uri + ","
        ids_count += 1
        if ids_count == max_requests:
            requests_chunks.append(concat_uris)
            concat_uris = ""
            ids_count = 0
    # For the last requests that didn't reach 100        
    requests_chunks.append(concat_uris)
    return requests_chunks

In [119]:
def get_auido_feat(requests_chunks):
    total_audio_feat = []

    for chunk in tqdm(requests_chunks):
        response = get_auido_feats_by_ids(chunk[0:-1])
        total_audio_feat.append(response.json()["audio_features"])
    
    return total_audio_feat

In [120]:
ahmeds_songs = pd.read_csv("MyData-Ali/Alis_songs.csv")

In [121]:
ahmeds_songs

,Unnamed: 0,endTime,artistName,trackName,msPlayed,uri
0,0,2022-01-15 14:42,The Paper Kites,Bloom (Online Bonus Track),34768,44alNkXsYnTyPnkMdohBcx
1,1,2022-01-15 14:45,Nordiska Piano,Solkysst,140482,0flcIAMaspYd4r5KIoZJ5L
2,2,2022-01-15 15:36,Niall Byrne,Howthian,95117,7Ifj1BA2mFHr1FmCxV9p1M
3,3,2022-01-16 19:14,Niall Byrne,Howthian,34011,7Ifj1BA2mFHr1FmCxV9p1M
4,4,2022-01-16 19:21,Tom Waits,Hold On,333625,5vjLSffimiIP26QG5WcN2K
...,...,...,...,...,...,...
2324,2324,2022-12-22 22:06,Bigsam,بحارب فيها,1537,0XDtC0f0gsLPPVvbFiu07q
2325,2325,2022-12-22 22:17,Bigsam,واسي,41633,4tkLzkjFOww06RWbNPInN2
2326,2326,2022-12-23 10:35,Afroto,FAR2 KHEBRA,36868,53BjuhvFCmv5T0VRhKuNrb
2327,2327,2022-12-23 10:37,Afroto,Brazil,79149,6cpTq72SH9QSocMTnGEKg5


In [122]:
unique_ids = ahmeds_songs["uri"].dropna().unique()
chunks = get_request_chunks(unique_ids)
songs_with_audio = get_auido_feat(chunks)

100%|█████████████████████████████████████████████| 9/9 [00:02<00:00,  3.60it/s]


In [123]:
audio_feat_dict = []

for requests_dict in songs_with_audio:
    for songs_dict in requests_dict:
        audio_feat_dict.append(pd.DataFrame([songs_dict]))

In [124]:
songs_audio = pd.concat(audio_feat_dict)

In [125]:
songs_audio.to_csv("MyData-Ali/songs_audio.csv")

In [81]:
added_to_playlist = pd.read_json("TechnicalData/MyData/ZineEvent.json")

In [82]:
added_to_playlist #[ added_to_playlist.message_source_end== "playlist_recommendations"]

,message_http_referer,message_referrer,timestamp_utc,context_time,message_context,message_error_code,message_feature,message_market,message_step,message_type
0,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 17:12:34.216000+00:00,2022-12-23 17:12:34.216,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_NO_PRICE_CHANGE
1,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 18:02:22.071000+00:00,2022-12-23 18:02:22.071,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_NO_PRICE_CHANGE
2,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 18:02:22.063000+00:00,2022-12-23 18:02:22.063,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_INITIAL
3,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 20:45:10.395000+00:00,2022-12-23 20:45:10.395,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_INITIAL
4,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 17:14:40.804000+00:00,2022-12-23 17:14:40.804,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_INITIAL
5,https://www.spotify.com/de/account/overview/?u...,https%3A%2F%2Faccounts.spotify.com%2F,2022-11-22 12:23:04.771000+00:00,2022-11-22 12:23:04.771,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_NO_PRICE_CHANGE
6,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 17:14:40.810000+00:00,2022-12-23 17:14:40.810,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_NO_PRICE_CHANGE
7,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 17:12:34.211000+00:00,2022-12-23 17:12:34.211,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_INITIAL
8,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 20:59:48.344000+00:00,2022-12-23 20:59:48.344,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_NO_PRICE_CHANGE
9,https://www.spotify.com/de/account/overview/?u...,None,2022-12-23 17:14:33.294000+00:00,2022-12-23 17:14:33.294,plan-section,NaN,feature.pricechange,de,INFO,IMPRESSION_NO_PRICE_CHANGE
